In [1]:
import nltk; nltk.download('stopwords')

!python3 -m spacy download en
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip && unzip mallet-2.0.8
!pip install pyLDAvis

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
--2019-12-08 12:40:43--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  7.38MB/s    in 2.1s    

2019-12-08 12:40:45 (7.38 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
   creating: mallet-2.0.8/
   creating: mallet-2.0.8/bin/
  inflating:

In [0]:
import re
import os
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [0]:
from google.colab import drive
drive.mount("/content/drive/")
os.chdir("./drive/My Drive/Colab Notebooks/Cleaned/")
os.getcwd()

In [0]:
def mk_folder_name(filename):
  return "./Files/"+filename

In [0]:
import numpy as np
txtfiles = np.array(os.listdir('.'))
totaldata = {}
for i in range (5555,5565):
  #path=mk_folder_name(txtfiles[i])
  #print (path)
  #!ls ./Files/
  #!mkdir -p path
  txtfile = open("./"+txtfiles[i])
  datastring = txtfile.read()
  data = np.array(datastring.split('\n'))
  totaldata[txtfiles[i]]= data[:-1]

totaldata

**Data Pre-Processing**

1) Tokenization
2) Stop Words Removal
3) Making Bigrams and Trigrams
4)Lemmatization


In [0]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

dictok = {}
for key in totaldata:
  data_words = list(sent_to_words(totaldata[key]))
  dictok[key] = data_words

for key, value in dictok.items():
  print (key," : ",value)

www.lowcarb-nocarb.com.txtGenerated.txt  :  [['take', 'the', 'protection', 'of', 'your', 'personal', 'data', 'very', 'seriously', 'treat', 'your', 'personal', 'data', 'confidentially', 'and', 'according', 'to', 'the', 'eu', 'general', 'data', 'protection', 'regulations', 'as', 'set', 'out', 'in', 'this', 'privacy', 'statement'], ['the', 'use', 'of', 'my', 'website', 'is', 'normally', 'possible', 'without', 'giving', 'personal', 'data', 'so', 'far', 'as', 'supplying', 'your', 'personal', 'data', 'on', 'our', 'pages', 'eg', 'name', 'address', 'or', 'mail', 'addresses', 'this', 'is', 'always', 'done', 'on', 'voluntary', 'basis', 'wherever', 'possible', 'these', 'data', 'will', 'not', 'be', 'passed', 'on', 'to', 'third', 'parties', 'without', 'your', 'prior', 'consent', 'hate', 'spam'], ['would', 'like', 'to', 'point', 'out', 'that', 'data', 'transmission', 'over', 'the', 'internet', 'for', 'example', 'by', 'mail', 'communication', 'can', 'constitute', 'security', 'risk', 'complete', 'prot

In [0]:
dicbg = {}
dictg = {}
for key in dictok:
  bigram = gensim.models.Phrases(dictok[key], min_count=1, threshold=7) # higher threshold fewer phrases.
  trigram = gensim.models.Phrases(bigram[dictok[key]],min_count=1, threshold=7)  

  # Faster way to get a sentence clubbed as a trigram/bigram
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)
  dicbg[key]=bigram_mod
  dictg[key]=trigram_mod
  # See trigram example
  #print(trigram_mod[bigram_mod[dictok[key][3]]])

for key in dictg:
  print (key," : ",end = " ")
  for index in dictok[key]:
    print (dictg[key][dicbg[key][index]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


www.lowcarb-nocarb.com.txtGenerated.txt  :  ['take', 'the', 'protection_of', 'your_personal_data', 'very', 'seriously', 'treat', 'your_personal_data', 'confidentially', 'and', 'according_to_the', 'eu', 'general', 'data_protection', 'regulations', 'as', 'set', 'out', 'in', 'this', 'privacy_statement']
['the_use_of', 'my', 'website', 'is', 'normally', 'possible', 'without', 'giving', 'personal_data', 'so', 'far', 'as', 'supplying', 'your_personal_data', 'on_our_pages', 'eg', 'name', 'address', 'or', 'mail', 'addresses', 'this_is', 'always', 'done', 'on', 'voluntary', 'basis', 'wherever', 'possible', 'these_data', 'will_not_be', 'passed', 'on', 'to_third_parties', 'without_your_prior', 'consent', 'hate', 'spam']
['would_like', 'to_point', 'out_that', 'data', 'transmission', 'over', 'the', 'internet', 'for_example', 'by', 'mail', 'communication', 'can', 'constitute', 'security', 'risk', 'complete', 'protection_of', 'the', 'data', 'from', 'access', 'by', 'third_parties', 'is_not', 'possible

In [0]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(key,texts):
    return [dicbg[key][doc] for doc in texts]

def make_trigrams(key,texts):
    return [dictg[key][dicbg[key][doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


In [0]:
dicns = {}
dicbgd = {}
dictgd = {}
dicl = {}
nlp = spacy.load('en', disable=['parser', 'ner'])
for key in dictok:
  dicns[key] = remove_stopwords(dictok[key])
  dicbgd[key] = make_bigrams(key,dicns[key])
  dictgd[key] = make_trigrams(key,dicns[key])  
  dicl[key] = lemmatization(dictgd[key], allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

for key in dicns:
  print (key," : ",dicns[key])
print ("##########################################################")
for key in dictgd:
  print (key," : ",dictgd[key])
print ("##########################################################")
for key in dicl:
  print (key," : ",dicl[key])

www.lowcarb-nocarb.com.txtGenerated.txt  :  [['take', 'protection', 'personal', 'data', 'seriously', 'treat', 'personal', 'data', 'confidentially', 'according', 'eu', 'general', 'data', 'protection', 'regulations', 'set', 'privacy', 'statement'], ['website', 'normally', 'possible', 'without', 'giving', 'personal', 'data', 'far', 'supplying', 'personal', 'data', 'pages', 'eg', 'name', 'address', 'mail', 'addresses', 'always', 'done', 'voluntary', 'basis', 'wherever', 'possible', 'data', 'passed', 'third', 'parties', 'without', 'prior', 'consent', 'hate', 'spam'], ['would', 'like', 'point', 'data', 'transmission', 'internet', 'example', 'mail', 'communication', 'constitute', 'security', 'risk', 'complete', 'protection', 'data', 'access', 'third', 'parties', 'possible'], ['chalking', 'success', 'uses', 'called', 'cookies', 'site', 'serve', 'make', 'web', 'page', 'user', 'friendly', 'effective', 'safer', 'small', 'text', 'files', 'stored', 'computer', 'cookies', 'used', 'us', 'deleted', 'h

In [0]:

dicids = {}
diccorp = {}
for key in dictok:
  dicids[key] = corpora.Dictionary(dicl[key])
  diccorp[key] = [dicids[key].doc2bow(text) for text in dicl[key]]


**LDA TOPIC MODEL**

In [0]:
dicldas = {}
for key in dictok:
  dicldas[key] = gensim.models.ldamodel.LdaModel(corpus=diccorp[key],
                                            id2word=dicids[key],
                                            num_topics=5, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=10,
                                            passes=10,
                                            alpha = 'auto', 
                                            per_word_topics=True)

In [0]:
for key in dictok:
  print('\nPerplexity: ', dicldas[key].log_perplexity(diccorp[key]))
  coherence_model_lda = CoherenceModel(model=dicldas[key], texts=dicl[key], dictionary=dicids[key], coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nCoherence Score: ', coherence_lda)
  print ("######################################################")


Perplexity:  -6.314096419772376

Coherence Score:  0.4385227005971986
######################################################

Perplexity:  -6.0392290095516135

Coherence Score:  0.3715068745158924
######################################################

Perplexity:  -5.855750703155877

Coherence Score:  0.4223074517381552
######################################################

Perplexity:  -5.601792894328551

Coherence Score:  0.4419076607224966
######################################################

Perplexity:  -5.520964869418302

Coherence Score:  0.3814815315451839
######################################################

Perplexity:  -5.219999885942676

Coherence Score:  0.6273132814694052
######################################################

Perplexity:  -5.3945953180760515

Coherence Score:  0.3901638529394759
######################################################

Perplexity:  -5.940556521987236

Coherence Score:  0.44452241171711326
############################################

**Topics Generated**
(Topics for each paragraphs of a file are generated)

In [0]:
dicvis = {}
for key in dictok:
  pprint (dicldas[key].print_topics())
  pyLDAvis.enable_notebook()
  dicvis[key] = pyLDAvis.gensim.prepare(dicldas[key], diccorp[key], dicids[key])

[(0,
  '0.051*"youtube" + 0.038*"pinterest" + 0.037*"instagram" + '
  '0.027*"particular" + 0.019*"direct_connection" + 0.014*"browser_provides" + '
  '0.013*"log" + 0.013*"would_like" + 0.012*"point" + 0.012*"button"'),
 (1,
  '0.074*"information" + 0.028*"datum" + 0.027*"right" + 0.024*"child" + '
  '0.024*"age" + 0.018*"request" + 0.017*"reserve" + 0.015*"send" + '
  '0.014*"case" + 0.013*"collect"'),
 (2,
  '0.123*"website" + 0.049*"site" + 0.035*"visit" + 0.028*"cookie" + '
  '0.026*"page" + 0.024*"may" + 0.022*"use" + 0.020*"link" + 0.019*"server" + '
  '0.019*"datum"'),
 (3,
  '0.080*"facebook" + 0.039*"plugin" + 0.029*"lowcarb_nocarb" + '
  '0.028*"purpose" + 0.020*"time" + 0.018*"google_analytic" + 0.017*"regard" + '
  '0.016*"browser" + 0.015*"find" + 0.015*"usa"'),
 (4,
  '0.039*"privacy_policy" + 0.030*"third_party" + 0.026*"privacy" + '
  '0.025*"link" + 0.017*"protect" + 0.014*"policy" + 0.014*"track" + '
  '0.013*"advertising" + 0.012*"privacy_statement" + 0.011*"well"')

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


[(0,
  '0.100*"website" + 0.081*"may" + 0.069*"cookie" + 0.038*"advertising" + '
  '0.033*"purpose" + 0.032*"service" + 0.024*"track" + 0.023*"use" + '
  '0.021*"browser" + 0.018*"provide"'),
 (1,
  '0.038*"visit" + 0.026*"ad" + 0.022*"third_party" + 0.019*"set" + '
  '0.019*"visitor" + 0.019*"google_analytic" + 0.018*"user" + '
  '0.015*"internet_browser" + 0.015*"anonymous_data" + 0.012*"base"'),
 (2,
  '0.052*"information" + 0.046*"website" + 0.039*"may" + '
  '0.036*"privacy_policy" + 0.030*"personal_information" + 0.028*"submit" + '
  '0.028*"email" + 0.019*"newsletter" + 0.019*"click" + 0.015*"access"'),
 (3,
  '0.028*"dog" + 0.027*"amazon" + 0.027*"program" + 0.025*"link" + 0.023*"com" '
  '+ 0.019*"pet" + 0.019*"new" + 0.019*"good" + 0.019*"share" + 0.019*"owner"'),
 (4,
  '0.054*"website" + 0.051*"time" + 0.040*"information" + 0.020*"age" + '
  '0.020*"child" + 0.020*"please_contact" + 0.019*"collect" + 0.015*"effort" + '
  '0.013*"include" + 0.012*"facebook_instagram"')]


/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


[(0,
  '0.036*"information" + 0.027*"use" + 0.019*"website" + 0.019*"cookie" + '
  '0.019*"visitor" + 0.011*"site" + 0.011*"user" + 0.011*"analytic" + '
  '0.011*"computer" + 0.011*"tool"'),
 (1,
  '0.087*"site" + 0.079*"service" + 0.058*"user" + 0.046*"may" + '
  '0.044*"policy" + 0.040*"cheap_software" + 0.030*"order" + 0.030*"change" + '
  '0.029*"provide" + 0.024*"website"'),
 (2,
  '0.021*"information" + 0.020*"protect" + 0.020*"update" + 0.015*"email" + '
  '0.015*"partner" + 0.015*"content" + 0.013*"include" + 0.010*"processing" + '
  '0.010*"personal_information" + 0.010*"data"'),
 (3,
  '0.053*"user" + 0.040*"site" + 0.027*"may" + 0.023*"cookie" + '
  '0.021*"information" + 0.020*"email_address" + 0.020*"mailing" + '
  '0.020*"send" + 0.013*"include" + 0.012*"activity"'),
 (4,
  '0.085*"link" + 0.061*"site" + 0.040*"information" + '
  '0.036*"personally_identifiable" + 0.032*"user" + 0.031*"third_party" + '
  '0.024*"associate" + 0.024*"datum" + 0.019*"visitor" + 0.019*"aggreg

...

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


[(0,
  '0.043*"agreement" + 0.030*"user" + 0.027*"service" + 0.020*"company" + '
  '0.015*"term" + 0.014*"right" + 0.014*"law" + 0.013*"partner" + '
  '0.012*"shall" + 0.011*"agree"'),
 (1,
  '0.073*"service" + 0.045*"user" + 0.019*"user_agrees" + 0.016*"provide" + '
  '0.013*"company_partner" + 0.012*"account" + 0.011*"obtain" + '
  '0.010*"information" + 0.009*"third_party_service_provider" + 0.009*"make"'),
 (2,
  '0.031*"service" + 0.020*"user" + 0.019*"material" + 0.017*"include" + '
  '0.013*"agree" + 0.013*"result" + 0.013*"warranty" + 0.012*"damage" + '
  '0.012*"user_acknowledge" + 0.012*"datum"'),
 (3,
  '0.054*"site" + 0.044*"urban" + 0.041*"engage" + 0.041*"consumer" + '
  '0.041*"relevant" + 0.041*"abundance" + 0.021*"visit" + 0.017*"information" '
  '+ 0.012*"website" + 0.009*"web"'),
 (4,
  '0.063*"content" + 0.051*"company_partner" + 0.045*"contain" + '
  '0.037*"third_party_service_provider" + 0.028*"user" + 0.026*"may" + '
  '0.020*"provision" + 0.015*"liable" + 0.014

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


[(0,
  '0.022*"need" + 0.018*"service" + 0.017*"account" + 0.016*"choose" + '
  '0.015*"provide" + 0.013*"litfire_publishing" + 0.013*"include" + '
  '0.013*"information" + 0.013*"publishing" + 0.013*"medium"'),
 (1,
  '0.028*"use" + 0.022*"protect" + 0.022*"make" + 0.022*"confidentiality" + '
  '0.020*"datum" + 0.019*"credit_card" + 0.017*"address" + 0.017*"payment" + '
  '0.017*"contact" + 0.016*"information"'),
 (2,
  '0.043*"post" + 0.036*"mail" + 0.027*"may" + 0.026*"website" + '
  '0.024*"privacy_policy" + 0.022*"collect" + 0.019*"content" + '
  '0.019*"information" + 0.019*"testimonial" + 0.018*"company"'),
 (3,
  '0.038*"website" + 0.038*"information" + 0.020*"site" + '
  '0.019*"personal_information" + 0.018*"litfire" + 0.018*"page" + '
  '0.016*"link" + 0.016*"sale" + 0.016*"merger_acquisition" + 0.013*"contain"'),
 (4,
  '0.066*"cookie" + 0.028*"website" + 0.022*"page" + 0.020*"fill" + '
  '0.014*"block" + 0.014*"browser_setting" + 0.014*"purpose" + '
  '0.014*"auto_fill" + 

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


Move Mouse on the circles. the bigger circle denotes the best topics. and the words on the right side denotes the keywords which formulated the topic

In [0]:
dicvis['www.uamodels.net.txtGenerated.txt']

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.124903  0.015388       1        1  62.232624
0      0.136836  0.012213       2        1  31.638973
3     -0.005993 -0.055864       3        1   5.884776
4     -0.002811  0.014229       4        1   0.134838
2     -0.003129  0.014034       5        1   0.108786, topic_info=   Category      Freq            Term     Total  loglift  logprob
39  Default  1.000000  beauty_product  1.000000  30.0000  30.0000
47  Default  1.000000        customer  1.000000  29.0000  29.0000
36  Default  1.000000        analysis  1.000000  28.0000  28.0000
34  Default  1.000000       aggregate  1.000000  27.0000  27.0000
5   Default  1.000000          wonder  1.000000  26.0000  26.0000
..      ...       ...             ...       ...      ...      ...
20   Topic5  0.001221           order  1.608742  -0.3601  -4.4998
17   Topic5  0.001221     information  1.748773  -0.4436  -4.4998
39   Topic5  0.001221  beauty_product  1.889361  -0.5209  -4.4998
31   Topic5  0.001221    uamodels_net  1.749089  -0.4438  -4.4998
24   Topic5  0.001221         provide  1.609000  -0.3603  -4.4998

[239 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
6         2  1.028836     access
7         2  1.028648    address
8         2  1.028755      admin
34        1  1.058594  aggregate
35        1  0.898650      allow
...     ...       ...        ...
32        2  1.028683       user
33        2  1.028966      valid
81        1  0.898962    visitor
82        1  0.899086     wisely
5         1  0.735765     wonder

[81 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 5, 3])

In [0]:
#mallet_path = './mallet-2.0.8/bin/'
#os.chdir(mallet_path)
#print (os.getcwd())
#dicldamt = {}
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
#for key in dictok:

  dicldamt[key] = gensim.models.wrappers.LdaMallet('mallet', corpus=diccorp[key], num_topics=5, id2word=dicids[key])

/content/drive/My Drive/Colab Notebooks/Cleaned/mallet-2.0.8/bin


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CalledProcessError: ignored